In [25]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [65]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 7

5


In [66]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=264,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                  

In [67]:
df = pd.read_csv("../data/ufc-master.csv")

In [68]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [69]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [70]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [71]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4056
4056
4056


In [72]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,label
200,Alex Caceres,Steven Peterson,-115,-105,86.956522,95.238095,2019-07-20,"San Antonio, Texas, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
201,Raquel Pennington,Irene Aldana,115,-135,115.000000,74.074074,2019-07-20,"San Antonio, Texas, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,0
202,Sam Alvey,Klidson Abreu,145,-165,145.000000,60.606061,2019-07-20,"San Antonio, Texas, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,1
203,Roxanne Modafferi,Jennifer Maia,110,-130,110.000000,76.923077,2019-07-20,"San Antonio, Texas, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,1
204,Ray Borg,Gabriel Silva,-185,160,54.054054,160.000000,2019-07-20,"San Antonio, Texas, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4251,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,1
4252,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
4253,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
4254,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,label
0,Dustin Poirier,Dan Hooker,-225,180,44.444444,180.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,Red,0
1,Mike Perry,Mickey Gall,-315,235,31.746032,235.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
2,Brendan Allen,Kyle Daukaus,-305,235,32.786885,235.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,0
3,Gian Villante,Maurice Greene,180,-230,180.000000,43.478261,2020-06-27,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,1
4,Luis Pena,Khama Worthy,-250,195,40.000000,195.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Aleksei Oleinik,Walt Harris,130,-150,130.000000,66.666667,2019-07-20,"San Antonio, Texas, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,1
196,Greg Hardy,Juan Adams,-105,-115,95.238095,86.956522,2019-07-20,"San Antonio, Texas, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
197,James Vick,Dan Hooker,105,-125,105.000000,80.000000,2019-07-20,"San Antonio, Texas, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,1
198,Alexander Hernandez,Francisco Trinaldo,-250,210,40.000000,210.000000,2019-07-20,"San Antonio, Texas, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,0


In [73]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [74]:
models[1][model_num]

'GaussianNB(priors=None, var_smoothing=1e-09)'

In [75]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [56]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [57]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [58]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [59]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_06_20_20
GaussianNB(priors=None, var_smoothing=1e-09)
['R_win_by_Decision_Unanimous', 'B_win_by_Submission', 'R_Weight_lbs', 'sig_str_dif', 'B_Height_cms', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'height_dif', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
0


In [17]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [18]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [19]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

Current best score is: 17382.67238600922
Feature: B_Reach_cms Score: 18604.28903559756
Feature: R_win_by_Decision_Majority Score: 19158.58256272941
The best feature was R_win_by_Decision_Majority.  It scored 19158.58256272941
Current best score is: 19158.58256272941
Feature: B_win_by_KO/TKO Score: 20117.399302408026
Feature: ko_dif Score: 20501.85756335953
The best feature was ko_dif.  It scored 20501.85756335953
Current best score is: 20501.85756335953
NO IMPROVEMENT
FINAL BEST SCORE: 20501.85756335953

model_4
GaussianNB(priors=None, var_smoothing=1e-09)
['ko_dif', 'R_win_by_Decision_Majority', 'R_longest_win_streak', 'longest_win_streak_dif', 'reach_dif', 'avg_sub_att_dif', 'R_win_by_Decision_Unanimous', 'B_win_by_Submission', 'R_Weight_lbs', 'sig_str_dif', 'B_Height_cms', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'height_dif', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
0



(3224, 20)

(3224,)

(3224, 2)

(200, 20)

...

(200, 16)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-225,180,0.009459,0.990541,0
1,-315,235,0.972789,0.027211,0
2,-305,235,0.989996,0.010004,0
3,180,-230,0.085513,0.914487,1
4,-250,195,0.713676,0.286324,1
...,...,...,...,...,...
195,130,-150,0.822298,0.177702,1
196,-105,-115,0.570434,0.429566,0
197,105,-125,0.533024,0.466976,1
198,-250,210,0.904101,0.095899,0



          Number of matches: 200
          Number of bets: 190
          Number of winning bets: 123
          Number of losing bets: 67
          Number of underdog bets: 70
          Number of underdog wins: 34
          Number of underdog losses: 36
          Number of Favorite bets: 112
          Number of favorite wins: 83
          Number of favorite losses: 29
          Number of even bets: 8
          Number of even wins: 6
          Number of even losses: 2
          Profit: 2726.4064572035136
          Profit per bet: 14.349507669492176
          Profit per match: 13.632032286017568
          
          
HI
HI
HI
HI
HI


In [60]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3224, 14)

(3224,)

(3224, 2)

(200, 14)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-225,180,0.719835,0.280165,0
1,-315,235,0.902719,0.097281,0
2,-305,235,0.989299,0.010701,0
3,180,-230,0.083414,0.916586,1
4,-250,195,0.675461,0.324539,1
...,...,...,...,...,...
195,130,-150,0.437889,0.562111,1
196,-105,-115,0.595720,0.404280,0
197,105,-125,0.511191,0.488809,1
198,-250,210,0.821491,0.178509,0



          Number of matches: 200
          Number of bets: 189
          Number of winning bets: 122
          Number of losing bets: 67
          Number of underdog bets: 69
          Number of underdog wins: 33
          Number of underdog losses: 36
          Number of Favorite bets: 113
          Number of favorite wins: 84
          Number of favorite losses: 29
          Number of even bets: 7
          Number of even wins: 5
          Number of even losses: 2
          Profit: 2561.251996927251
          Profit per bet: 13.551597867339954
          Profit per match: 12.806259984636256
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [61]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [62]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [63]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [64]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=264,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                  